Experiment settings

In [1]:
import simpy
import numpy as np
import pandas as pd
from numpy import random
import math
import matplotlib as plt
import seaborn as sns

RANDOM_SEED = 42
### problem paramters ####
order_time_mu = 2
order_time_min = 1
food_prepare_mu = 3
food_prepare_min = 1

lamb = 0.4
n_kitchen = 3
n_counter = 2
n_customer = 2000
SIM_TIME = 24 * 60 #12 hours

random.seed(RANDOM_SEED)

# helper functions, monitoring and pandas table


In [2]:
def round_down(n, decimals=1):
    multiplier = 10 ** decimals
    return abs(math.floor(n*multiplier + 0.5) / multiplier)

counter_total_service_times = 0
counter_total_idle_times = 0
env = simpy.Environment()
# ,"Clock Time","Interarrival Time","Service Time","Wait Time","Queue Length","Total Time In System"
column_names = ["Arrival Time","Queue time","Service Start Time","Food prepare start","Exit system time","Food Prepare Duration","Total Wait Time(Queue+Food)","Service Time","Total Time in System"]
result_fifo = np.zeros((n_customer,len(column_names)))
result_tick = np.zeros((n_customer,len(column_names)))


# Simulation setting
## A) FIFO Algorithim

In [3]:
class Counter(object):
#     Counters to take order
    def __init__(self,env,num_counter):
        self.env = env
        self.counter = simpy.Resource(env,num_counter)
        self.counter_waiting = 0
        self.service_start = None
        
    def take_order(self,cus,env,service_start):
        print("%s is placing order at counter %.2f" %(cus,service_start))
        time_taken_to_place_order = max(random.exponential(scale = order_time_mu),order_time_min)
        yield self.env.timeout(time_taken_to_place_order)
        print("Order of %s sent to kitchen at %.2f" %(cus, env.now))
        # Record idle counter and add to total count
        
        
    def receive_order(self,cus,env,resource,service_start,data):
        global counter_total_idle_times
        global counter_total_service_times
        idle_start = env.now
        with resource.kitchen.request() as my_turn:
            yield my_turn
            yield env.process(resource.prepare_food(cus,env,data))
            service_end = env.now
            print("%s collected the food at %.2f" %(cus, service_end))
            counter_total_idle_times += (service_end-idle_start)
            counter_total_service_times += (service_end-service_start)
            

class Kitchen(object):
    # Kitchen to prepare food
    def __init__(self,env,num_kitchen):
        self.env = env
        self.kitchen = simpy.Resource(env,num_kitchen)
    
    def prepare_food(self,cus,env,data):
        print("Kitchen is preparing food for %s at %.2f" %(cus, env.now))
        food_prepare_time = max(food_prepare_min,random.exponential(scale = food_prepare_mu))
        data[cus,5] = round_down(food_prepare_time)
        yield self.env.timeout(food_prepare_time)
        print("Cooked food for %s at %.2f" %(cus, env.now))

def customer(env, label, queue, kitchen, data):
#     the customer process arrive at the restaurant and request counter to take order
    label = label-1
    arrive_time = env.now
    print("%s entering the queue at %.2f"%(label,arrive_time))
#     data[label,0]=label
    data[label,0]= round_down(arrive_time)
    with queue.counter.request() as my_turn:
        yield my_turn
        service_start = env.now
        data[label,2] = round_down(service_start)
        queue_time = service_start - arrive_time
        data[label,1]= round_down(queue_time)
        # placing order at counter
        yield env.process(queue.take_order(label,env,service_start))
        # waiting order at counter
        prepare_food_start = env.now
        data[label,3] = round_down(prepare_food_start)
        # counter is idle now
        yield env.process(queue.receive_order(label,env,kitchen,service_start,data))
        # prepare_food_end = round_down(env.now)
        # counter_total_wait_times += round_down(prepare_food_end - prepare_food_start)
        # receive food from counter
        exit_time = env.now
        data[label,4] = round_down(exit_time)

        # total wait time
        data[label,6] = round_down(data[label,5]+data[label,1])
        # total service time
        data[label,7] = round_down(exit_time-service_start)
        # total time in system
        data[label,8] = round_down(exit_time-arrive_time)
        yield env.timeout(0)

## Start FIFO Simulation

In [4]:
counter = Counter(env,n_counter)
kitchen = Kitchen(env,n_kitchen)
# Simlating possion process for customer arrival
def customer_arrivals(env,res_counter):
    """Create new *customer* until the sim time reaches 120. with poisson process"""
    for i in range(n_customer):
        yield env.timeout(random.poisson(1/lamb))
        env.process(customer(env, i+1, res_counter, kitchen, result_fifo))

env.process(customer_arrivals(env,counter))
env.run(until=SIM_TIME)

0 entering the queue at 4.00
0 is placing order at counter 4.00
1 entering the queue at 5.00
1 is placing order at counter 5.00
2 entering the queue at 7.00
Order of 0 sent to kitchen at 8.02
Kitchen is preparing food for 0 at 8.02
3 entering the queue at 9.00
Cooked food for 0 at 9.02
0 collected the food at 9.02
2 is placing order at counter 9.02
Order of 2 sent to kitchen at 10.02
Kitchen is preparing food for 2 at 10.02
4 entering the queue at 11.00
5 entering the queue at 12.00
Order of 1 sent to kitchen at 12.01
Kitchen is preparing food for 1 at 12.01
Cooked food for 2 at 12.86
2 collected the food at 12.86
3 is placing order at counter 12.86
Cooked food for 1 at 14.17
1 collected the food at 14.17
4 is placing order at counter 14.17
Order of 3 sent to kitchen at 14.66
Kitchen is preparing food for 3 at 14.66
6 entering the queue at 15.00
Order of 4 sent to kitchen at 15.17
Kitchen is preparing food for 4 at 15.17
7 entering the queue at 16.00
Cooked food for 3 at 17.46
3 collec

## FIFO Result & Analysis

In [24]:
labels = [*range(1,n_customer+1)]
np_arr = np.array(result_fifo).reshape(n_customer,-1)
df_fifo=pd.DataFrame(data = np_arr,index=labels,columns=column_names)


,Arrival Time,Queue time,Service Start Time,Food prepare start,Exit system time,Food Prepare Duration,Total Wait Time(Queue+Food),Service Time,Total Time in System
525,1310.0,122.5,1432.5,1439.4,0.0,1.0,0.0,0.0,0.0
526,1316.0,119.4,1435.4,1437.2,0.0,10.1,0.0,0.0,0.0
527,1323.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
528,1325.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
529,1325.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# Converting to pandas
np_arr = np.array(result_fifo).reshape(n_customer,-1)
labels = [*range(1,n_customer+1)]
df_fifo=pd.DataFrame(data = np_arr,index=labels,columns=column_names)
# df_fifo

df_fifo=df_fifo.drop(df_fifo[df_fifo.iloc[:,-1]==0].index,axis=0) # remove unfinished customer

total_wait_time = df_fifo.iloc[:,6].sum()
total_service_time = df_fifo.iloc[:,7].sum()

total_time_in_system = df_fifo.iloc[:,8].sum()
print("")

In [29]:
# df_fifo.iloc[1:30,:]
df_fifo.iloc[-50:,:]
# sns.displot(df_fifo.iloc[:,5])
# sns.plot()

,Arrival Time,Queue time,Service Start Time,Food prepare start,Exit system time,Food Prepare Duration,Total Wait Time(Queue+Food),Service Time,Total Time in System
475,1186.0,113.6,1299.6,1301.9,1303.6,1.7,115.3,4.0,117.6
476,1188.0,115.6,1303.6,1304.6,1310.2,5.6,121.2,6.6,122.2
477,1190.0,114.7,1304.7,1312.6,1315.6,2.9,117.6,10.9,125.6
478,1193.0,117.2,1310.2,1312.0,1313.0,1.0,118.2,2.8,120.0
479,1196.0,117.0,1313.0,1320.9,1321.9,1.0,118.0,8.9,125.9
480,1201.0,114.6,1315.6,1316.6,1318.5,1.9,116.5,2.9,117.5
481,1203.0,115.5,1318.5,1321.0,1324.1,3.1,118.6,5.6,121.1
482,1205.0,116.9,1321.9,1324.1,1326.2,2.1,119.0,4.3,121.2
483,1207.0,117.1,1324.1,1325.1,1328.1,3.0,120.1,4.0,121.1
484,1210.0,116.2,1326.2,1327.2,1328.9,1.7,117.9,2.7,118.9


In [30]:
n_served = df_fifo.shape[0]

def display(i):
    print()
    print("%d Customers served"%n_served)
    print("Total Simulation Time=> %.2f Minutes" % SIM_TIME)
    print("Counter Total Idle Time => %.2f Minutes" % (counter_total_idle_times))
    print()
    print("Average Waiting Time => %.2f Minutes" % (total_wait_time / i))
    print("Average Service Time => %.2f Minutes" % (total_service_time / i))
    print("Average Time Spent In System => %.2f Minutes" % (total_time_in_system / i))

display(n_served)


524 Customers served
Total Simulation Time=> 1440.00 Minutes
Counter Total Idle Time => 1620.60 Minutes

Average Waiting Time => 88.27 Minutes
Average Service Time => 5.46 Minutes
Average Time Spent In System => 90.63 Minutes
